In [ ]:
import cv2
import numpy as np
from collections import deque
import time
import matplotlib.pyplot as plt
import os
from sklearn.metrics import silhouette_score

def resize_frame(frame, scale_percent=30):
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    return cv2.resize(frame, (width, height))

def analyze_region(frame, x, y, width, sig=15):
    frame_height, frame_width = frame.shape[:2]
    x = max(0, min(x, frame_width - width))
    y = max(0, min(y, frame_height - width))
    
    roi = frame[y:y+width, x:x+width].copy()
    blurred_roi = cv2.GaussianBlur(roi, (25,25), 0)
    
    # Quantize colors to reduce subtle variations
    # Divide by 16 and multiply back to reduce color space
    quantized_roi = (blurred_roi // 16) * 16
    
    lab_roi = cv2.cvtColor(blurred_roi, cv2.COLOR_BGR2LAB)
    
    pixel_vals = quantized_roi.reshape((-1, 3)).astype(np.float32)
    
    # More stringent criteria for convergence
    # Increased epsilon from 10 to 30 to require more significant difference
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1)
    n_clusters = 5
    
    # Add minimum distance between cluster centers
    min_distance = 30 # Adjust this value to control sensitivity
    
    while True:
        _, labels, centers = cv2.kmeans(pixel_vals, n_clusters, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
        
        # Check distances between all pairs of center
        centers_valid = True
        for i in range(len(centers)):
            for j in range(i + 1, len(centers)):
                dist = np.linalg.norm(centers[i] - centers[j])
                if dist < min_distance:
                    centers_valid = False
                    break
            if not centers_valid:
                break
                
        if centers_valid:
            break
            
        # If centers are too close, reduce number of clusters
        n_clusters -= 1
        if n_clusters < 1:
            n_clusters = 1
            break
    #if n_clusters > 1:
     #   sil_score = silhouette_score(pixel_vals, labels=labels.flatten())
    
    #print(f"sil_score: {sil_score}")
    #count pixel frequency per label
    counts = np.bincount(labels.flatten())
    # Increased threshold from 0.20 to 0.25 to require more significant color regions
    print(f"pixelvals: {len(pixel_vals)}")
    #significant cluster if it occupies greater than 10% of total pixels in frame
    significant_clusters = np.sum(counts > len(pixel_vals) * .10)
    
    centers = np.uint8(centers)
    segmented_data = centers[labels.flatten()]
    segmented_image = segmented_data.reshape(roi.shape)

    #print(f"Significant_clusters: {significant_clusters}, x: {x}, y: {y}")
    
    return significant_clusters, x, y, roi, blurred_roi, lab_roi, segmented_image

def visualize_processing_steps(frame, res_x, res_y, roi, blurred_roi, lab_roi, distinct_colors, kmeans_output, window_name="Processing Steps"):
    
    frame_height, frame_width = frame.shape[:2]
    roi_height, roi_width = roi.shape[:2]

    vis_width = frame_width * 6 
    visualization = np.zeros((frame_height, vis_width, 3), dtype=np.uint8)

    frame_with_roi = frame.copy()
    cv2.rectangle(frame_with_roi, 
                  (res_x, res_y),
                  (res_x + roi_width - 1, res_y + roi_height - 1),
                  (0, 0, 255), 2)  # Highlight square ROI
    visualization[:, :frame_width] = frame_with_roi

    roi_section = visualization[:, frame_width:frame_width * 2]
    roi_resized = cv2.resize(roi, (frame_width, frame_height))
    roi_section[:frame_height, :] = roi_resized

    gaussian_section = visualization[:, frame_width*2:frame_width * 3]
    gaussian_resized = cv2.resize(blurred_roi, (frame_width, frame_height))
    gaussian_section[:frame_height, :] = gaussian_resized

    lab_section = visualization[:, frame_width * 3:frame_width * 4]
    l_channel, a_channel, b_channel = cv2.split(lab_roi)
    l_norm = cv2.normalize(l_channel, None, 0, 255, cv2.NORM_MINMAX)
    a_norm = cv2.normalize(a_channel, None, 0, 255, cv2.NORM_MINMAX)
    b_norm = cv2.normalize(b_channel, None, 0, 255, cv2.NORM_MINMAX)
    lab_visual = cv2.merge([l_norm, a_norm, b_norm])
    lab_resized = cv2.resize(lab_visual, (frame_width, frame_height))
    lab_section[:frame_height, :] = lab_resized

    variation_section = visualization[:, frame_width * 4:frame_width * 5]
    lab_variation = np.std(lab_roi, axis=2)
    lab_variation = cv2.normalize(lab_variation, None, 0, 255, cv2.NORM_MINMAX)
    variation_map = cv2.applyColorMap(lab_variation.astype(np.uint8), cv2.COLORMAP_JET)
    variation_resized = cv2.resize(variation_map, (frame_width, frame_height))
    variation_section[:frame_height, :] = variation_resized

    kmeans_section = visualization[:, frame_width * 5:frame_width * 6]
    kmeans_resized = cv2.resize(kmeans_output, (frame_width, frame_height))
    kmeans_section[:frame_height, :] = kmeans_resized


    # Modified section for kmeans visualization
    kmeans_section = visualization[:, frame_width * 5:frame_width * 6]
    kmeans_resized = cv2.resize(kmeans_output, (frame_width, frame_height))
    kmeans_section[:frame_height, :] = kmeans_resized

    # Get unique colors and their positions from kmeans output
    unique_colors = np.unique(kmeans_output.reshape(-1, 3), axis=0)
    for color in unique_colors:
        # Find positions where this color appears
        mask = np.all(kmeans_resized == color, axis=2)
        positions = np.where(mask)
        if len(positions[0]) > 0:
            # Calculate center of mass for this color
            y_center = int(np.mean(positions[0]))
            x_center = int(np.mean(positions[1]))
            # Draw circle at the center position
            center_pos = (frame_width * 5 + x_center, y_center)
            cv2.circle(visualization, center_pos, 5, (255, 255, 255), -1)  # White fill
            cv2.circle(visualization, center_pos, 5, (0, 0, 0), 1)  # Black border

    # Modified section for kmeans visualization
    kmeans_section = visualization[:, frame_width * 5:frame_width * 6]
    kmeans_resized = cv2.resize(kmeans_output, (frame_width, frame_height))
    kmeans_section[:frame_height, :] = kmeans_resized

    # Get unique colors and their positions from kmeans output
    unique_colors = np.unique(kmeans_output.reshape(-1, 3), axis=0)
    centers = []  # Store centers for drawing connections
    
    # First pass: find and draw centers
    for color in unique_colors:
        # Find positions where this color appears
        mask = np.all(kmeans_resized == color, axis=2)
        positions = np.where(mask)
        if len(positions[0]) > 0:
            # Calculate center of mass for this color
            y_center = int(np.mean(positions[0]))
            x_center = int(np.mean(positions[1]))
            center_pos = (frame_width * 5 + x_center, y_center)
            centers.append(center_pos)
            # Draw circle at the center position
            cv2.circle(visualization, center_pos, 5, (255, 255, 255), -1)  # White fill
            cv2.circle(visualization, center_pos, 5, (0, 0, 0), 1)  # Black border
    
    # Second pass: draw lines and distances between centers
    for i in range(len(centers)):
        for j in range(i + 1, len(centers)):
            center1 = centers[i]
            center2 = centers[j]
            # Calculate Euclidean distance
            distance = np.sqrt((center1[0] - center2[0])**2 + (center1[1] - center2[1])**2)
            # Draw line between centers
            cv2.line(visualization, center1, center2, (0, 255, 0), 1)
            # Calculate midpoint for text
            mid_x = (center1[0] + center2[0]) // 2
            mid_y = (center1[1] + center2[1]) // 2
            # Draw distance value
 
            cv2.putText(visualization, f'{distance:.1f}', (mid_x, mid_y), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 1)  # Black text

    # Draw color centers as circles in the bottom right corner of kmeans section
    center_size = 30
    spacing = 40
    start_x = frame_width * 5 + 20
    start_y = frame_height - 40

    # Get unique colors from kmeans output
    unique_colors = np.unique(kmeans_output.reshape(-1, 3), axis=0)
    
    # Draw circles for each center color
    for i, color in enumerate(unique_colors):
        x = start_x + i * spacing
        cv2.circle(visualization, (x, start_y), center_size // 2, color.tolist(), -1)
        # Draw white border around circle
        cv2.circle(visualization, (x, start_y), center_size // 2, (255, 255, 255), 1)
        # Add distance to nearest center if more than one center exists
        if len(unique_colors) > 1:
            distances = [np.linalg.norm(color - other_color) for other_color in unique_colors if not np.array_equal(color, other_color)]
            min_dist = min(distances)
            cv2.putText(visualization, f'{min_dist:.0f}', (x - 10, start_y + 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.7
    font_thickness = 2
    def add_text_with_background(img, text, pos):
        (text_width, text_height), baseline = cv2.getTextSize(text, font, font_scale, font_thickness)
        cv2.rectangle(img, 
                      (pos[0] - 5, pos[1] - text_height - baseline - 5),
                      (pos[0] + text_width + 5, pos[1] + 5),
                      (0, 0, 0), -1)
        cv2.putText(img, text, pos, font, font_scale, (255, 255, 255), font_thickness)

    add_text_with_background(visualization, 'Original Frame', (10, 30))
    add_text_with_background(visualization, 'ROI', (frame_width + 10, 30))
    add_text_with_background(visualization, 'Gaussian', (frame_width * 2 + 10, 30))
    add_text_with_background(visualization, 'LAB Space', (frame_width * 3 + 10, 30))
    add_text_with_background(visualization, 'Color Variation', (frame_width * 5 + 10, 30))
    add_text_with_background(visualization, f'KMeans Output (Colors: {distinct_colors})', (frame_width * 4 + 10, 30))

    cv2.imshow(window_name, visualization)
    cv2.waitKey(1)
    return visualization

def detect_color_change(video_path, scale_percent=30, buffer_size=9, threshold=0.7, save_interval=30):

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file: {video_path}")
        return None, None

    ret, first_frame = cap.read()
    if ret:
        print(f"Initial frame shape: {first_frame.shape}")
        scaled_shape = resize_frame(first_frame, scale_percent).shape
        print(f"Scaled frame shape: {scaled_shape}")
        roi_height = int(scaled_shape[0] * 0.2)
        print(f"Expected ROI height: {roi_height}")
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Reset to start
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    frame_interval = max(1, int(fps / 5))
    expected_samples = total_frames // frame_interval
    
    print(f"\nVideo information:")
    print(f"Total frames: {total_frames}")
    print(f"Original FPS: {fps:.1f}")
    print(f"Analyzing every {frame_interval}th frame (3Hz)")
    print(f"Expected samples: {expected_samples}")
    
    color_counts = deque(maxlen=buffer_size)
    frame_number = 0
    samples_processed = 0
    last_update_time = time.time()
    last_save_time = time.time()
    os.makedirs('processing_steps', exist_ok=True)
    sig = -1
    
    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_number += 1
            
            if frame_number % frame_interval != 0:
                continue
            
            samples_processed += 1
            current_time = time.time()

            small_frame = resize_frame(frame, scale_percent)
            denoised = cv2.fastNlMeansDenoisingColored(small_frame, None, 5, 5, 3, 9)

            distinct_colors, res_x, res_y, roi, blurred_roi, lab_roi, kmeans_output = analyze_region(denoised, x=310, y=50, width=260, sig=sig)
            color_counts.append(distinct_colors)
            print(f"color_counts : {color_counts}")
            vis = visualize_processing_steps(denoised, res_x, res_y, roi, blurred_roi, lab_roi, distinct_colors, kmeans_output)
            
            if current_time - last_save_time > save_interval:
                save_path = f'processing_steps/frame_{frame_number}.jpg'
                cv2.imwrite(save_path, vis)
                last_save_time = current_time
            
            if current_time - last_update_time > 2:
                progress = (frame_number / total_frames) * 100
                elapsed_time = current_time - last_update_time
                samples_per_second = samples_processed / elapsed_time if elapsed_time > 0 else 0
                print(f"Progress: {progress:.1f}% (Frame {frame_number}/{total_frames}, "
                      f"Processing rate: {samples_per_second:.1f} Hz)")
                samples_processed = 0
                last_update_time = current_time
            
            if len(color_counts) == buffer_size:
                #np.mean(color_counts)
                #avg_colors = np.median(color_counts)
                avg_colors = np.bincount(color_counts)
                #avg_colors < 2
                if avg_colors[2] == buffer_size:
                    timestamp = frame_number / fps
                    print(f"\nColor change detected!")
                    print(f"Frame: {frame_number}/{total_frames} ({(frame_number/total_frames)*100:.1f}%)")
                    print(f"Timestamp: {timestamp:.2f} seconds")
                    print(f"Avg number of colors: {avg_colors}")
                    print(f"color_counts: {color_counts}")
                    print(f"")
                    
                    # Save final detection frame
                    cv2.imwrite('processing_steps/detection_frame.jpg', vis)
                    
                    cap.release()
                    cv2.destroyAllWindows()
                    return frame_number, timestamp

    
    except Exception as e:
        print(f"Error during processing: {str(e)}")
        cap.release()
        cv2.destroyAllWindows()
        raise
    
    print("\nProcessing complete - No clear color change point detected")
    cap.release()
    cv2.destroyAllWindows()
    return None, None



In [ ]:
file = open("testresults_jan24th.txt", "a")
path = os.open("TODO")

for video in path:
    frame_number, timestamp = detect_color_change(video)
    file.write(f"Frame number: {frame_number}, timestamp: {timestamp}, Video: {video}")


In [ ]:
import cv2
import numpy as np
from collections import deque
import time
import matplotlib.pyplot as plt
import os

def resize_frame(frame, scale_percent=30):
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    return cv2.resize(frame, (width, height))

def analyze_region(frame, x, y, width, threshold, clusters, min_distance):
    frame_height, frame_width = frame.shape[:2]
    x = max(0, min(x, frame_width - width))
    y = max(0, min(y, frame_height - width))
    
    roi = frame[y:y+width, x:x+width].copy()
    blurred_roi = cv2.GaussianBlur(roi, (25,25), 0)
    
    # Quantize colors to reduce subtle variations
    # Divide by 16 and multiply back to reduce color space
    quantized_roi = (blurred_roi // 16) * 16
    
    lab_roi = cv2.cvtColor(blurred_roi, cv2.COLOR_BGR2LAB)
    
    pixel_vals = quantized_roi.reshape((-1, 3)).astype(np.float32)
    
    # More stringent criteria for convergence
    # Increased epsilon from 10 to 30 to require more significant difference
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1)
    n_clusters = clusters
    
    # Add minimum distance between cluster centers
    #min_distance = 30 # Adjust this value to control sensitivity
    
    while True:
        _, labels, centers = cv2.kmeans(pixel_vals, n_clusters, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
        
        # Check distances between all pairs of center
        centers_valid = True
        for i in range(len(centers)):
            for j in range(i + 1, len(centers)):
                dist = np.linalg.norm(centers[i] - centers[j])
                if dist < min_distance:
                    centers_valid = False
                    break
            if not centers_valid:
                break
                
        if centers_valid:
            break
            
        # If centers are too close, reduce number of clusters
        n_clusters -= 1
        if n_clusters < 1:
            n_clusters = 1
            break
    
    #count pixel frequency per label
    counts = np.bincount(labels.flatten())
    # Increased threshold from 0.20 to 0.25 to require more significant color regions
    #print(f"pixelvals: {len(pixel_vals)}")
    #significant cluster if it occupies greater than 10% of total pixels in frame
    significant_clusters = np.sum(counts > len(pixel_vals) * threshold)
    
    centers = np.uint8(centers)
    segmented_data = centers[labels.flatten()]
    segmented_image = segmented_data.reshape(roi.shape)

    #print(f"Significant_clusters: {significant_clusters}, x: {x}, y: {y}")
    
    return significant_clusters, x, y, roi, blurred_roi, lab_roi, segmented_image

def visualize_processing_steps(frame, res_x, res_y, roi, blurred_roi, lab_roi, distinct_colors, kmeans_output, window_name="Processing Steps"):
    
    frame_height, frame_width = frame.shape[:2]
    roi_height, roi_width = roi.shape[:2]

    vis_width = frame_width * 6 
    visualization = np.zeros((frame_height, vis_width, 3), dtype=np.uint8)

    frame_with_roi = frame.copy()
    cv2.rectangle(frame_with_roi, 
                  (res_x, res_y),
                  (res_x + roi_width - 1, res_y + roi_height - 1),
                  (0, 0, 255), 2)  # Highlight square ROI
    visualization[:, :frame_width] = frame_with_roi

    roi_section = visualization[:, frame_width:frame_width * 2]
    roi_resized = cv2.resize(roi, (frame_width, frame_height))
    roi_section[:frame_height, :] = roi_resized

    gaussian_section = visualization[:, frame_width*2:frame_width * 3]
    gaussian_resized = cv2.resize(blurred_roi, (frame_width, frame_height))
    gaussian_section[:frame_height, :] = gaussian_resized

    lab_section = visualization[:, frame_width * 3:frame_width * 4]
    l_channel, a_channel, b_channel = cv2.split(lab_roi)
    l_norm = cv2.normalize(l_channel, None, 0, 255, cv2.NORM_MINMAX)
    a_norm = cv2.normalize(a_channel, None, 0, 255, cv2.NORM_MINMAX)
    b_norm = cv2.normalize(b_channel, None, 0, 255, cv2.NORM_MINMAX)
    lab_visual = cv2.merge([l_norm, a_norm, b_norm])
    lab_resized = cv2.resize(lab_visual, (frame_width, frame_height))
    lab_section[:frame_height, :] = lab_resized

    variation_section = visualization[:, frame_width * 4:frame_width * 5]
    lab_variation = np.std(lab_roi, axis=2)
    lab_variation = cv2.normalize(lab_variation, None, 0, 255, cv2.NORM_MINMAX)
    variation_map = cv2.applyColorMap(lab_variation.astype(np.uint8), cv2.COLORMAP_JET)
    variation_resized = cv2.resize(variation_map, (frame_width, frame_height))
    variation_section[:frame_height, :] = variation_resized

    kmeans_section = visualization[:, frame_width * 5:frame_width * 6]
    kmeans_resized = cv2.resize(kmeans_output, (frame_width, frame_height))
    kmeans_section[:frame_height, :] = kmeans_resized


    # Modified section for kmeans visualization
    kmeans_section = visualization[:, frame_width * 5:frame_width * 6]
    kmeans_resized = cv2.resize(kmeans_output, (frame_width, frame_height))
    kmeans_section[:frame_height, :] = kmeans_resized

    # Get unique colors and their positions from kmeans output
    unique_colors = np.unique(kmeans_output.reshape(-1, 3), axis=0)
    for color in unique_colors:
        # Find positions where this color appears
        mask = np.all(kmeans_resized == color, axis=2)
        positions = np.where(mask)
        if len(positions[0]) > 0:
            # Calculate center of mass for this color
            y_center = int(np.mean(positions[0]))
            x_center = int(np.mean(positions[1]))
            # Draw circle at the center position
            center_pos = (frame_width * 5 + x_center, y_center)
            cv2.circle(visualization, center_pos, 5, (255, 255, 255), -1)  # White fill
            cv2.circle(visualization, center_pos, 5, (0, 0, 0), 1)  # Black border

    # Modified section for kmeans visualization
    kmeans_section = visualization[:, frame_width * 5:frame_width * 6]
    kmeans_resized = cv2.resize(kmeans_output, (frame_width, frame_height))
    kmeans_section[:frame_height, :] = kmeans_resized

    # Get unique colors and their positions from kmeans output
    unique_colors = np.unique(kmeans_output.reshape(-1, 3), axis=0)
    centers = []  # Store centers for drawing connections
    
    # First pass: find and draw centers
    for color in unique_colors:
        # Find positions where this color appears
        mask = np.all(kmeans_resized == color, axis=2)
        positions = np.where(mask)
        if len(positions[0]) > 0:
            # Calculate center of mass for this color
            y_center = int(np.mean(positions[0]))
            x_center = int(np.mean(positions[1]))
            center_pos = (frame_width * 5 + x_center, y_center)
            centers.append(center_pos)
            # Draw circle at the center position
            cv2.circle(visualization, center_pos, 5, (255, 255, 255), -1)  # White fill
            cv2.circle(visualization, center_pos, 5, (0, 0, 0), 1)  # Black border
    
    # Second pass: draw lines and distances between centers
    for i in range(len(centers)):
        for j in range(i + 1, len(centers)):
            center1 = centers[i]
            center2 = centers[j]
            # Calculate Euclidean distance
            distance = np.sqrt((center1[0] - center2[0])**2 + (center1[1] - center2[1])**2)
            # Draw line between centers
            cv2.line(visualization, center1, center2, (0, 255, 0), 1)
            # Calculate midpoint for text
            mid_x = (center1[0] + center2[0]) // 2
            mid_y = (center1[1] + center2[1]) // 2
            # Draw distance value
 
            cv2.putText(visualization, f'{distance:.1f}', (mid_x, mid_y), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 1)  # Black text

    # Draw color centers as circles in the bottom right corner of kmeans section
    center_size = 30
    spacing = 40
    start_x = frame_width * 5 + 20
    start_y = frame_height - 40

    # Get unique colors from kmeans output
    unique_colors = np.unique(kmeans_output.reshape(-1, 3), axis=0)
    
    # Draw circles for each center color
    for i, color in enumerate(unique_colors):
        x = start_x + i * spacing
        cv2.circle(visualization, (x, start_y), center_size // 2, color.tolist(), -1)
        # Draw white border around circle
        cv2.circle(visualization, (x, start_y), center_size // 2, (255, 255, 255), 1)
        # Add distance to nearest center if more than one center exists
        if len(unique_colors) > 1:
            distances = [np.linalg.norm(color - other_color) for other_color in unique_colors if not np.array_equal(color, other_color)]
            min_dist = min(distances)
            cv2.putText(visualization, f'{min_dist:.0f}', (x - 10, start_y + 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.7
    font_thickness = 2
    def add_text_with_background(img, text, pos):
        (text_width, text_height), baseline = cv2.getTextSize(text, font, font_scale, font_thickness)
        cv2.rectangle(img, 
                      (pos[0] - 5, pos[1] - text_height - baseline - 5),
                      (pos[0] + text_width + 5, pos[1] + 5),
                      (0, 0, 0), -1)
        cv2.putText(img, text, pos, font, font_scale, (255, 255, 255), font_thickness)

    add_text_with_background(visualization, 'Original Frame', (10, 30))
    add_text_with_background(visualization, 'ROI', (frame_width + 10, 30))
    add_text_with_background(visualization, 'Gaussian', (frame_width * 2 + 10, 30))
    add_text_with_background(visualization, 'LAB Space', (frame_width * 3 + 10, 30))
    add_text_with_background(visualization, 'Color Variation', (frame_width * 4 + 10, 30))
    add_text_with_background(visualization, f'KMeans Output (Colors: {distinct_colors})', (frame_width * 5 + 10, 30))

    cv2.imshow(window_name, visualization)
    cv2.waitKey(1)
    return visualization

def detect_color_change(video_path, scale_percent=30, buffer_size=9, threshold=0.7, save_interval=30, clusters=7, min_distance=20):

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file: {video_path}")
        return None, None

    ret, first_frame = cap.read()
    if ret:
        print(f"Initial frame shape: {first_frame.shape}")
        scaled_shape = resize_frame(first_frame, scale_percent).shape
        print(f"Scaled frame shape: {scaled_shape}")
        roi_height = int(scaled_shape[0] * 0.2)
        print(f"Expected ROI height: {roi_height}")
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Reset to start
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    frame_interval = max(1, int(fps / 5))
    expected_samples = total_frames // frame_interval
    
    '''
    print(f"\nVideo information:")
    print(f"Total frames: {total_frames}")
    print(f"Original FPS: {fps:.1f}")
    print(f"Analyzing every {frame_interval}th frame (3Hz)")
    print(f"Expected samples: {expected_samples}")'''
    
    color_counts = deque(maxlen=buffer_size)
    frame_number = 0
    samples_processed = 0
    last_update_time = time.time()
    last_save_time = time.time()
    os.makedirs('processing_steps', exist_ok=True)
    
    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_number += 1
            
            if frame_number % frame_interval != 0:
                continue
            
            samples_processed += 1
            current_time = time.time()

            small_frame = resize_frame(frame, scale_percent)
            denoised = cv2.fastNlMeansDenoisingColored(small_frame, None, 5, 5, 3, 9)

            distinct_colors, res_x, res_y, roi, blurred_roi, lab_roi, kmeans_output = analyze_region(denoised, x=310, y=50, width=260, threshold=threshold, clusters=clusters, min_distance=min_distance)
            color_counts.append(distinct_colors)
            #print(f"color_counts : {color_counts}")
            #vis = visualize_processing_steps(denoised, res_x, res_y, roi, blurred_roi, lab_roi, distinct_colors, kmeans_output)
            
            if current_time - last_save_time > save_interval:
                #save_path = f'processing_steps/frame_{frame_number}.jpg'
                #cv2.imwrite(save_path, vis)
                last_save_time = current_time
            
            if current_time - last_update_time > 2:
                progress = (frame_number / total_frames) * 100
                elapsed_time = current_time - last_update_time
                samples_per_second = samples_processed / elapsed_time if elapsed_time > 0 else 0
                #print(f"Progress: {progress:.1f}% (Frame {frame_number}/{total_frames}, "
                     # f"Processing rate: {samples_per_second:.1f} Hz)")
                samples_processed = 0
                last_update_time = current_time
            
            if len(color_counts) == buffer_size:
                #np.mean(color_counts)
                avg_colors = np.median(color_counts)
                avg_colors = np.bincount(color_counts)
                #avg_colors < 2
                if avg_colors[2] == buffer_size:
                    timestamp = frame_number / fps
                    print(f"\nColor change detected!")
                    print(f"Frame: {frame_number}/{total_frames} ({(frame_number/total_frames)*100:.1f}%)")
                    print(f"Timestamp: {timestamp:.2f} seconds")
                    print(f"Avg number of colors: {avg_colors}")
                    print(f"color_counts: {color_counts}")
                    print(f"")
                    
                    # Save final detection frame
                    #cv2.imwrite('processing_steps/detection_frame.jpg', vis)
                    
                    cap.release()
                    cv2.destroyAllWindows()
                    return frame_number, timestamp
    
    except Exception as e:
        print(f"Error during processing: {str(e)}")
        cap.release()
        cv2.destroyAllWindows()
        raise
    
    #print("\nProcessing complete - No clear color change point detected")
    cap.release()
    cv2.destroyAllWindows()
    return -1, -1


#things we want to modify: % significant cluster (10-30), n_clusters (4-8), min_distance(10-50)